In [ ]:
import pandas as pd

# paths to data
train_df = pd.read_csv("data/kaggle/train.csv")
test_df  = pd.read_csv("data/kaggle/test.csv")

print("Train shape:", train_df.shape)
print(train_df.head())
print(train_df.isnull().sum())


In [ ]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print(train_df[label_cols].mean())


In [ ]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", " ", text)           # strip HTML tags
    text = re.sub(r"https?://\S+", " ", text)    # strip URLs
    text = re.sub(r"[^a-z\s]", " ", text)        # keep only letters & spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

train_df['comment_text'] = train_df['comment_text'].apply(clean_text)
test_df['comment_text']  = test_df['comment_text'].apply(clean_text)


In [ ]:

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    train_df['comment_text'],
    train_df[label_cols],
    test_size=0.1,
    random_state=42
)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    max_features=10_000, 
    stop_words='english', 
    ngram_range=(1,2)
)

X_train_bow = vectorizer.fit_transform(X_train)
X_val_bow   = vectorizer.transform(X_val)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

# initialize
base_clf = LogisticRegression(C=1.0, max_iter=200)
model = OneVsRestClassifier(base_clf, n_jobs=-1)

# fit
model.fit(X_train_bow, y_train)

# predict probabilities on validation
y_val_pred = model.predict_proba(X_val_bow)

# compute AUC for each label
for i, label in enumerate(label_cols):
    auc = roc_auc_score(y_val[label], y_val_pred[:, i])
    print(f"{label} AUC: {auc:.4f}")


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

# initialize
base_clf = LogisticRegression(C=1.0, max_iter=200)
model = OneVsRestClassifier(base_clf, n_jobs=-1)

# fit
model.fit(X_train_bow, y_train)

# predict probabilities on validation
y_val_pred = model.predict_proba(X_val_bow)

# compute AUC for each label
for i, label in enumerate(label_cols):
    auc = roc_auc_score(y_val[label], y_val_pred[:, i])
    print(f"{label} AUC: {auc:.4f}")


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
    ('vect', CountVectorizer(stop_words='english')),
    ('clf', OneVsRestClassifier(LogisticRegression(), n_jobs=-1))
])

param_grid = {
    'vect__max_features': [5_000, 10_000, 20_000],
    'vect__ngram_range': [(1,1), (1,2)],
    'clf__estimator__C': [0.1, 1.0, 10.0]
}

grid = GridSearchCV(
    pipeline,
    param_grid,
    scoring='roc_auc',
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)


In [ ]:
# retrain on full data
full_vect = CountVectorizer(max_features=10_000, stop_words='english', ngram_range=(1,2))
X_full = full_vect.fit_transform(train_df['comment_text'])

final_clf = OneVsRestClassifier(LogisticRegression(C=1.0, max_iter=200), n_jobs=-1)
final_clf.fit(X_full, train_df[label_cols])

# predict on test
X_test = full_vect.transform(test_df['comment_text'])
probs = final_clf.predict_proba(X_test)

sub = pd.DataFrame(probs, columns=label_cols)
sub.insert(0, 'id', test_df['id'])


In [ ]:

print("First 5 predictions:")
print(sub.head(), "\n")

print("Summary statistics of predicted probabilities:")
print(sub.describe().T, "\n")

import pandas as pd
from sklearn.metrics import roc_auc_score, classification_report

test_labels = pd.read_csv("data/kaggle/test_labels.csv")      # adjust path
label_cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']


mask = (test_labels[label_cols] != -1).all(axis=1)
scored = test_labels[mask].set_index('id')

# align predictions to the scored IDs
preds = sub.set_index('id').loc[scored.index, label_cols]

# compute per-label ROC AUC
print("ROC AUC per label:")
for col in label_cols:
    auc = roc_auc_score(scored[col], preds[col])
    print(f"  {col:15s}: {auc:.4f}")

# classification report @ 0.5 threshold
binary_preds = (preds >= 0.5).astype(int)
print("\nClassification Report (threshold=0.5):")
print(classification_report(scored, binary_preds, target_names=label_cols))
